<a href="https://colab.research.google.com/github/leoguti/route-compare/blob/main/notebooks/notebooks00_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import gspread
from google.colab import auth
import google.auth # Import google.auth

# Function to extract sheet ID and GID
def get_sheet_ids(sheet_url):
    match = re.search(r"/d/([a-zA-Z0-9_-]+)", sheet_url)
    if not match:
        raise ValueError("No se pudo encontrar el ID de la hoja de cálculo en la URL.")
    sheet_id = match.group(1)

    gid_match = re.search(r"gid=(\d+)", sheet_url)
    gid = gid_match.group(1) if gid_match else "0"
    return sheet_id, gid

LISTADO_TRUJILLO_V2_URL = "https://docs.google.com/spreadsheets/d/1DqplLS5iLnz3oHqtJCCdOvpUyhWGnL5VzeyaGhb2VSA/edit?gid=0#gid=0"
DATOS_TRUJILLO_2024_URL = "https://docs.google.com/spreadsheets/d/1XKLTLMK2hgWXyGf3Qqn9n-KnHrSgSgWokk9GB8BT6lg/edit?gid=0#gid=0"

# Authenticate Google Colab to access Google Sheets
auth.authenticate_user()
# Get credentials from the authenticated user
credentials, project_id = google.auth.default()
# Initialize gspread client with the obtained credentials
gc = gspread.Client(auth=credentials);

# Read LISTADO_TRUJILLO_V2
print("Reading LISTADO_TRUJILLO_V2...")
listado_v2_sheet_id, listado_v2_gid = get_sheet_ids(LISTADO_TRUJILLO_V2_URL)
listado_v2_spreadsheet = gc.open_by_key(listado_v2_sheet_id)
listado_v2_worksheet = listado_v2_spreadsheet.get_worksheet_by_id(int(listado_v2_gid)) # gspread needs GID as int
listado_v2_data = listado_v2_worksheet.get_all_records()
listado_v2_df = pd.DataFrame.from_records(listado_v2_data)

# Read DATOS_TRUJILLO_2024
print("Reading DATOS_TRUJILLO_2024...")
datos_2024_sheet_id, datos_2024_gid = get_sheet_ids(DATOS_TRUJILLO_2024_URL)
datos_2024_spreadsheet = gc.open_by_key(datos_2024_sheet_id)
datos_2024_worksheet = datos_2024_spreadsheet.get_worksheet_by_id(int(datos_2024_gid))

# Get all values from the worksheet to manually handle headers
all_values = datos_2024_worksheet.get_all_values()

if all_values:
    # Assume the first row is the header
    headers = all_values[0]
    # Process headers to ensure uniqueness, especially for empty strings
    processed_headers = []
    seen_headers = {}
    for h in headers:
        original_h = h.strip() if h else '' # Clean up and handle truly empty strings
        if original_h in seen_headers:
            seen_headers[original_h] += 1
            processed_headers.append(f"{original_h}_{seen_headers[original_h]}")
        else:
            seen_headers[original_h] = 0 # Initialize counter for first occurrence
            processed_headers.append(original_h)

    # The rest of the rows are data
    datos_2024_data = all_values[1:]
    datos_2024_df = pd.DataFrame(datos_2024_data, columns=processed_headers)
else:
    datos_2024_df = pd.DataFrame() # Create an empty DataFrame if no data

print("Successfully read both Google Sheets.")
display(listado_v2_df.head())
display(datos_2024_df.head())

In [177]:
# 1) Buscar columnas que contengan la palabra "normalizado" (sin importar mayúsculas/minúsculas)

cols_normalizado_listado = [c for c in listado_v2_df.columns if "normalizado" in c.lower()]
cols_normalizado_datos = [c for c in datos_2024_df.columns if "normalizado" in c.lower()]

print("Columnas con 'normalizado' en LISTADO_TRUJILLO_V2:")
print(cols_normalizado_listado if cols_normalizado_listado else "  (No se encontraron columnas)")

print("\nColumnas con 'normalizado' en DATOS_TRUJILLO_2024:")
print(cols_normalizado_datos if cols_normalizado_datos else "  (No se encontraron columnas)")

# 2) Explorar cada columna encontrada

def explorar_cols_normalizado(df, cols, nombre_df):
    print(f"\n===== Exploración en {nombre_df} =====")
    for col in cols:
        print(f"\n--- Columna: {col} ---")
        print("Tipo de datos:", df[col].dtype)
        print("Cantidad de filas:", len(df))
        print("Nulos:", df[col].isna().sum())
        print("Valores únicos:", df[col].nunique())

        # Mostrar algunos ejemplos de valores
        print("\nEjemplos de valores (primeros 10 no nulos):")
        print(df[col].dropna().head(10).tolist())

        # Distribución básica (primeros 10 valores más frecuentes)
        print("\nValores más frecuentes:")
        print(df[col].value_counts(dropna=True).head(10))
        print("-" * 40)

# Llamar a la función de exploración para cada dataframe
if cols_normalizado_listado:
    explorar_cols_normalizado(listado_v2_df, cols_normalizado_listado, "LISTADO_TRUJILLO_V2")

if cols_normalizado_datos:
    explorar_cols_normalizado(datos_2024_df, cols_normalizado_datos, "DATOS_TRUJILLO_2024")


Columnas con 'normalizado' en LISTADO_TRUJILLO_V2:
['Normalizado']

Columnas con 'normalizado' en DATOS_TRUJILLO_2024:
['Normalizado']

===== Exploración en LISTADO_TRUJILLO_V2 =====

--- Columna: Normalizado ---
Tipo de datos: object
Cantidad de filas: 126
Nulos: 0
Valores únicos: 81

Ejemplos de valores (primeros 10 no nulos):
['M-01 C', 'M-02 A', 'M-03 B', 'M-03 B', 'M-04 A', 'M-05 H', 'M-05 H', 'M-06 B', 'M-07 V', 'M-08 A']

Valores más frecuentes:
Normalizado
M-35 D        7
C-11 T1       5
M-23 B        4
C-15 P1       4
C-28 C        3
C-42 B2       3
M-34 A        3
M-15 C2       2
M-32 A1-B1    2
M-22 F        2
Name: count, dtype: int64
----------------------------------------

===== Exploración en DATOS_TRUJILLO_2024 =====

--- Columna: Normalizado ---
Tipo de datos: object
Cantidad de filas: 152
Nulos: 0
Valores únicos: 131

Ejemplos de valores (primeros 10 no nulos):
['', 'C-01 B', 'C-02 H', 'C-03 JJ', 'C-04 XS', 'C-05 B1', 'C-06 K', 'C-07 Z1', 'C-08 O', 'C-09 E']

Valores

In [179]:
set_listado = set(listado_v2_df["Normalizado"].astype(str).str.strip())
set_datos = set(datos_2024_df["Normalizado"].astype(str).str.strip())

print("Valores en ambos:")
print(len(set_listado & set_datos))
print(set_listado & set_datos)

print("\nValores en listado V2 que NO están en datos 2024:")
print(len(set_listado - set_datos))

print("\nValores en datos 2024 que NO están en listado V2:")
print(len(set_datos - set_listado))


Valores en ambos:
74
{'C-12 B', 'C-20 UI', 'C-39 S', 'M-09 BC', 'C-05 B1', 'M-14 C3', 'M-12 C', 'C-25 S2', 'M-24 A', 'M-35 D', 'M-13 C1', 'M-08 A', 'C-27 B', 'M-20 C', 'C-06 K', 'C-17 F', 'C-14 S', 'M-01 C', 'C-29 A', 'C-26 U', 'C-34 G', 'C-43 A', 'C-13 D', 'C-38 S4', 'M-06 B', 'C-42 B2', 'M-07 V', 'M-16 C', 'M-31 A', 'C-07 Z1', 'M-03 B', 'C-41 Z2', 'C-22 M', 'C-16 I', 'C-45 C', 'C-18 P', 'C-02 H', 'C-44 T2', 'M-18 C1', 'M-11 D', 'M-34 A', 'C-36 Y', 'C-10 M', 'M-26 B', 'C-11 T1', 'C-35 S3', 'C-40 W', 'C-09 E', 'C-23 S1', 'C-21 R', 'M-21 A', 'C-04 XS', 'C-28 C', 'M-19 A', 'M-33 B', 'M-05 H', 'M-27 AB', 'M-36 LV', 'M-17 C2', 'M-29 B', 'C-01 B', 'M-10 B1', 'M-25 A', 'M-04 A', 'C-37 R', 'M-23 B', 'M-30 D', 'M-15 C2', 'C-03 JJ', 'C-19 NN', 'C-08 O', 'M-22 F', 'M-28 A', 'M-02 A'}

Valores en listado V2 que NO están en datos 2024:
7

Valores en datos 2024 que NO están en listado V2:
57


In [181]:
# 1. Crear conjuntos para comparación rápida
set_datos = set(datos_2024_df["Normalizado"].astype(str).str.strip())

# 2. Extraer la columna Normalizado del maestro
normalizado_listado = listado_v2_df["Normalizado"].astype(str).str.strip()

# 3. Crear una nueva columna con el resultado del match exacto
listado_v2_df["Match_Exacto"] = normalizado_listado.apply(
    lambda x: "MATCH" if x in set_datos else ""
)

# 4. Preparar datos para actualizar en Google Sheets
# Google Sheets requiere una lista vertical: [[valor], [valor], ...]
columna_J_valores = [[v] for v in listado_v2_df["Match_Exacto"].tolist()]

# 5. Escribir en la columna J (columna 10) del sheet maestro
listado_v2_worksheet.update(
    range_name=f"J1:J{len(columna_J_valores)}",
    values=columna_J_valores
)

print("✔️ Columna J del LISTADO_TRUJILLO_V2 actualizada con coincidencias exactas.")


✔️ Columna J del LISTADO_TRUJILLO_V2 actualizada con coincidencias exactas.


In [183]:
import pandas as pd

# 1. Normalizar/limpiar la columna Normalizado en ambos dataframes
listado_v2_df["Normalizado_trim"] = listado_v2_df["Normalizado"].astype(str).str.strip()
datos_2024_df["Normalizado_trim"] = datos_2024_df["Normalizado"].astype(str).str.strip()

# 2. Crear conteos por código (ignorando vacíos después del strip)
master_counts = (
    listado_v2_df[listado_v2_df["Normalizado_trim"] != ""]
    ["Normalizado_trim"]
    .value_counts()
)

datos_counts = (
    datos_2024_df[datos_2024_df["Normalizado_trim"] != ""]
    ["Normalizado_trim"]
    .value_counts()
)

# 3. Función para clasificar el estado de cada código del maestro
def clasificar_estado(cod):
    if pd.isna(cod):
        return ""
    cod = str(cod).strip()
    if cod == "":
        # ignoramos valores vacíos
        return ""

    n_m = master_counts.get(cod, 0)  # apariciones en maestro
    n_d = datos_counts.get(cod, 0)   # apariciones en datos 2024

    # Sin ninguna aparición en datos 2024
    if n_d == 0:
        return "NO_match_en_2024"

    # A partir de aquí, n_d > 0 y n_m >= 1
    if n_m == 1 and n_d == 1:
        return "MATCH_1a1"
    elif n_m > 1 and n_d == 1:
        return "MATCH_master_multi_2024_1"
    elif n_m == 1 and n_d > 1:
        return "MATCH_master_1_2024_multi"
    elif n_m > 1 and n_d > 1:
        return "MATCH_multi_en_ambos"

    # En teoría no deberíamos llegar aquí, pero dejamos un fallback vacío
    return ""

# 4. Aplicar la clasificación al maestro
listado_v2_df["Estado_match"] = listado_v2_df["Normalizado_trim"].apply(clasificar_estado)

# (Opcional) Ver un resumen en pantalla
print("Resumen de estados en el maestro:")
print(listado_v2_df["Estado_match"].value_counts(dropna=False))

# 5. Preparar los valores para escribir en la columna J de Google Sheets
# J1 será el encabezado y desde J2 los estados de cada fila de datos
valores_J = [["Estado_match"]] + [[v] for v in listado_v2_df["Estado_match"].tolist()]
num_filas = len(valores_J)  # incluye encabezado

# 6. Actualizar la columna J completa (desde J1 hasta J{num_filas})
listado_v2_worksheet.update(f"J1:J{num_filas}", valores_J)

print("✔️ Columna J del LISTADO_TRUJILLO_V2 actualizada con 'Estado_match'.")


Resumen de estados en el maestro:
Estado_match
MATCH_1a1                    46
MATCH_master_multi_2024_1    39
MATCH_multi_en_ambos         26
NO_match_en_2024             13
MATCH_master_1_2024_multi     2
Name: count, dtype: int64


/tmp/ipython-input-2317589197.py:62: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  listado_v2_worksheet.update(f"J1:J{num_filas}", valores_J)


✔️ Columna J del LISTADO_TRUJILLO_V2 actualizada con 'Estado_match'.


In [184]:
# 1. Identificar la columna ID en datos_2024_df (por si el encabezado tiene espacios o mayúsculas)
id_col = None
for c in datos_2024_df.columns:
    if c.strip().lower() == "id":
        id_col = c
        break

if id_col is None:
    raise ValueError("No se encontró una columna llamada 'ID' en DATOS_TRUJILLO_2024.")

# 2. Normalizar columnas relevantes en datos_2024_df
datos_2024_df["Normalizado_trim"] = datos_2024_df["Normalizado_trim"].astype(str).str.strip()
datos_2024_df[id_col] = datos_2024_df[id_col].astype(str).str.strip()

# 3. Crear un mapa: Normalizado_trim -> ID (primer ID encontrado)
id_map = (
    datos_2024_df[datos_2024_df["Normalizado_trim"] != ""]
    .groupby("Normalizado_trim")[id_col]
    .first()
    .to_dict()
)

# 4. Para cada fila del maestro, si es MATCH_1a1 buscar el ID; si no hay, "no operativa"
def obtener_id_osm(row):
    estado = row.get("Estado_match", "")
    if estado != "MATCH_1a1":
        # Solo llenamos para MATCH_1a1; otros se quedan vacíos
        return ""

    cod = str(row.get("Normalizado_trim", "")).strip()
    if cod == "":
        return ""

    osm_id = id_map.get(cod, "").strip()
    if osm_id:
        return osm_id
    else:
        return "no operativa"

listado_v2_df["ID_OSM_2024"] = listado_v2_df.apply(obtener_id_osm, axis=1)

# (Opcional) Ver un pequeño resumen en pantalla
print(listado_v2_df[["Normalizado_trim", "Estado_match", "ID_OSM_2024"]].head(20))

# 5. Escribir en la columna K del maestro (K1 encabezado, K2+ datos)
valores_K = [["ID_OSM_2024"]] + [[v] for v in listado_v2_df["ID_OSM_2024"].tolist()]
num_filas_K = len(valores_K)

listado_v2_worksheet.update(f"K1:K{num_filas_K}", valores_K)

print("✔️ Columna K del LISTADO_TRUJILLO_V2 actualizada con 'ID_OSM_2024' para casos MATCH_1a1.")


   Normalizado_trim               Estado_match   ID_OSM_2024
0            M-01 C                  MATCH_1a1      17527405
1            M-02 A                  MATCH_1a1      17522670
2            M-03 B       MATCH_multi_en_ambos              
3            M-03 B       MATCH_multi_en_ambos              
4            M-04 A                  MATCH_1a1  no operativa
5            M-05 H  MATCH_master_multi_2024_1              
6            M-05 H  MATCH_master_multi_2024_1              
7            M-06 B                  MATCH_1a1      17551463
8            M-07 V                  MATCH_1a1      17434299
9            M-08 A                  MATCH_1a1      17554401
10          M-09 BC                  MATCH_1a1      17554410
11          M-10 B1  MATCH_master_multi_2024_1              
12          M-10 B1  MATCH_master_multi_2024_1              
13           M-11 D                  MATCH_1a1  no operativa
14           M-12 C                  MATCH_1a1      17554572
15          M-36 LV     

/tmp/ipython-input-261472298.py:49: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  listado_v2_worksheet.update(f"K1:K{num_filas_K}", valores_K)


✔️ Columna K del LISTADO_TRUJILLO_V2 actualizada con 'ID_OSM_2024' para casos MATCH_1a1.


In [186]:
# Asumimos que listado_v2_df ya está cargado con las columnas J y K

col_estado = "Estado_match"     # Columna J
col_k = "ID_OSM_2024"           # Columna K

# 1. Contar estados en columna J
conteo_estados = listado_v2_df[col_estado].value_counts(dropna=False)

print("===== CONSOLIDADO POR ESTADO_MATCH (columna J) =====")
for estado, cantidad in conteo_estados.items():
    print(f"{estado}: {cantidad}")

print("\n")

# 2. Contar no operativas (K = 'no operativa')
no_operativas = (listado_v2_df[col_k].astype(str).str.strip().str.lower() == "no operativa").sum()
print(f"Rutas marcadas como NO OPERATIVAS en K: {no_operativas}")

# 3. Contar rutas con ID OSM válido (tienen relación OSM)
con_id_osm = listado_v2_df[col_k].astype(str).str.strip()
con_id_osm = con_id_osm[ (con_id_osm != "") & (con_id_osm.str.lower() != "no operativa") ]
print(f"Rutas con ID OSM (operativas con relación): {len(con_id_osm)}")

# 4. Total analizado
print(f"Total de rutas analizadas en el maestro: {len(listado_v2_df)}")


===== CONSOLIDADO POR ESTADO_MATCH (columna J) =====
MATCH_1a1: 46
MATCH_master_multi_2024_1: 39
MATCH_multi_en_ambos: 26
NO_match_en_2024: 13
MATCH_master_1_2024_multi: 2


Rutas marcadas como NO OPERATIVAS en K: 17
Rutas con ID OSM (operativas con relación): 29
Total de rutas analizadas en el maestro: 126
